## Import libraries

In [1]:
!pip install pytorch-ignite
!pip install torcheval
!pip install optuna
!pip install plotly

In [2]:
COLAB = False

In [3]:
if COLAB:
    import sys
    import os
    from google.colab import drive

    # Mount Google Driveroo
    drive.mount('/content/drive')

    # Add the path to the Python module
    root_dir = '/content/drive/MyDrive/text_summarization'
    sys.path.append(os.path.join(root_dir, 'src'))
    sys.path.append(os.path.join(root_dir, 'src', 'utils'))
else:
    from pathlib import Path
    root_dir = Path.cwd().parent
root_dir

PosixPath('/home/jovyan/text_summarization')

In [1]:
import pandas as pd
import numpy as np
import os
import re
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
stopwords = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/johannes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
from utils.preprocessing import preprocessing_pipeline, get_data_distribution
from utils.processing import processing_pipeline, processing_pipeline_packed
from utils.data_structures import create_packed_dataloader
from train_model import main, tuning
import utils.inference as inference
from optuna.visualization import (plot_optimization_history,
                                  plot_param_importances, plot_slice)

/Users/johannes/opt/anaconda3/envs/Language_Engineering_pip/lib/python3.11/site-packages/ignite/handlers/checkpoint.py:16: DeprecationWarning: `TorchScript` support for functional optimizers is deprecated and will be removed in a future PyTorch release. Consider using the `torch.compile` optimizer instead.
  from torch.distributed.optim import ZeroRedundancyOptimizer


In [3]:
PROCESSING = True
PREPROCESSING = True
HYP_TUNING = True
legacy = False

In [4]:
name = "WikiHow"

In [6]:
raw_dir = os.path.join(root_dir, "raw_data", name)
if legacy:
    dataset_dir = os.path.join(root_dir, "data", name)
    figures_dir = os.path.join(root_dir, "figures", name)
    os.makedirs(dataset_dir, exist_ok=True)
    os.makedirs(figures_dir, exist_ok=True)
else:
    dataset_dir = os.path.join(root_dir, "data_packed", name)
    figures_dir = os.path.join(root_dir, "figures_packed", name)
    os.makedirs(dataset_dir, exist_ok=True)
    os.makedirs(figures_dir, exist_ok=True)

## Get the data

In [7]:
dataset_df = pd.read_csv(os.path.join(raw_dir, "wikihowSep.csv"))

In [8]:
csv_name = "wikihow_data"

## Preprocess the data

In [9]:
if PREPROCESSING:
    subset_size = 0.2 # Change later back to 50%
    preprocessing_pipeline(dataset_df, stopwords, dataset_dir, csv_name, subset_size = subset_size, start_token = "SOS ", end_token = " EOS")

/Users/johannes/Studium/Master/6. Semester/Language engineering/Project/text_summarization/src/utils/preprocessing.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name] = start_token + df[column_name] + end_token
/Users/johannes/Studium/Master/6. Semester/Language engineering/Project/text_summarization/src/utils/preprocessing.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[f'length_{column_name}'] = df[column_name].apply(lambda x: len(x.split()))
/Users/johannes/Studium/Master/6. Seme

In [10]:
if legacy:
    dataset_df = pd.read_csv(os.path.join(root_dir, "data", name, f'{csv_name}.csv'))
else:
    dataset_df = pd.read_csv(os.path.join(root_dir, "data_packed", name, f'{csv_name}.csv'))
    
dataset_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 269126 entries, 0 to 269125
Data columns (total 4 columns):
 #   Column          Non-Null Count   Dtype 
---  ------          --------------   ----- 
 0   summary         269126 non-null  object
 1   text            269126 non-null  object
 2   length_summary  269126 non-null  int64 
 3   length_text     269126 non-null  int64 
dtypes: int64(2), object(2)
memory usage: 8.2+ MB


In [11]:
dataset_df.head()

,summary,text,length_summary,length_text
0,SOS put yourself out there EOS,SOS its simpleif you dont put yourself into ne...,6,40
1,SOS hold your finger down to open closing opti...,SOS on some devices running dolphin you can al...,10,31
2,SOS combine beeswax and turpentine to create a...,SOS mix 1 part beeswax with 3 parts turpentine...,10,22
3,SOS let up on the clutch while pushing down on...,SOS in order to get moving lift your left foot...,13,172
4,SOS locate the song you would like to rate EOS,SOS using the scroll wheel locate the song tha...,10,15


## Get distribution of the data

In [ ]:
get_data_distribution(dataset_df, figures_dir, "wikihow")

/Users/johannes/Studium/Master/6. Semester/Language engineering/Project/text_summarization/src/utils/preprocessing.py:99: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


## Process the data

In [12]:
load_tokenizer = False

In [13]:
if PROCESSING and legacy:
    print('legacy used')
    processing_pipeline(dataset_dir, csv_name, load_tokenizer = load_tokenizer)
elif PROCESSING and not legacy:
    processing_pipeline_packed(dataset_dir, csv_name, load_tokenizer=load_tokenizer)

Max length of text: 243
Max length of summary: 30
SOS token index: 1
EOS token index: 2
UNK token index: 3
PAD token index: 0
Number of Samples in X_train: 188388
Number of Samples in X_val: 48442
Number of Samples in X_test: 32296
Number of Samples in y_train: 188388
Number of Samples in y_val: 48442
Number of Samples in y_test: 32296
Vocabulary size: 283401
Saving x_train_list.pt...
x_train_list.pt dtype: torch.int64, shape: torch.Size([74])
Saving x_val_list.pt...
x_val_list.pt dtype: torch.int64, shape: torch.Size([173])
Saving x_test_list.pt...
x_test_list.pt dtype: torch.int64, shape: torch.Size([18])
Saving y_train_list.pt...
y_train_list.pt dtype: torch.int64, shape: torch.Size([7])
Saving y_val_list.pt...
y_val_list.pt dtype: torch.int64, shape: torch.Size([5])
Saving y_test_list.pt...
y_test_list.pt dtype: torch.int64, shape: torch.Size([16])


### Test the processing

In [14]:
test_decoding = True

In [15]:
if test_decoding:
    import torch
    import pickle
    import random
    
    def decode_data(text_ids, index2word, EOS_token):
        """
        Converts the text ids to words using the index2word mapping.
        """
        if text_ids.dim() > 1:
            text_ids = text_ids.view(-1)  # Flatten to 1D
    
        decoded_words = []
        for idx in text_ids:
            # Ensure idx is a scalar
            if isinstance(idx, torch.Tensor):
                idx = idx.item()
            if idx == EOS_token:
                decoded_words.append('EOS')
                break
            decoded_words.append(index2word.get(idx, 'UNK'))
    
        return " ".join(decoded_words)
    
    if legacy:
        X_train = torch.load(os.path.join(dataset_dir, "x_train.pt"))
        y_train = torch.load(os.path.join(dataset_dir, "y_train.pt"))
        train_dataloader = torch.utils.data.DataLoader(
            torch.utils.data.TensorDataset(X_train, y_train),
            batch_size=1,
            shuffle=False,
            )
    else:
        X_train = torch.load(os.path.join(dataset_dir, "x_train_list.pt"))
        y_train = torch.load(os.path.join(dataset_dir, "y_train_list.pt"))
        train_dataloader = create_packed_dataloader(X_train, y_train, pad_id=0, batch_size=1, shuffle=False)
    
    
    with open(os.path.join(dataset_dir, 'feature_tokenizer.pickle'), 'rb') as handle:
            feature_tokenizer = pickle.load(handle)
    EOS_token = feature_tokenizer.word2index.get("EOS", 2)
    
    nb_decoding_test = 10
    count_test = 0
    random_list = random.sample(range(len(train_dataloader)), nb_decoding_test)
    if legacy:
        for i, data in enumerate(train_dataloader):
            if i in random_list:
                input_tensor, target_tensor = data
                print('Input: {}'.format(decode_data(input_tensor[0], feature_tokenizer.index2word, EOS_token)))
                print('Target: {}'.format(decode_data(target_tensor[0], feature_tokenizer.index2word, EOS_token)))
                print('-----------------------------------')
                count_test += 1
            if count_test == nb_decoding_test:
                break
    else:
        for i, data in enumerate(train_dataloader):
            if i in random_list:
                input_tensor, input_lengths, target_tensor, target_lengths = data
                print('Input: {}'.format(decode_data(input_tensor[0], feature_tokenizer.index2word, EOS_token)))
                print('Target: {}'.format(decode_data(target_tensor[0], feature_tokenizer.index2word, EOS_token)))
                print('-----------------------------------')
                count_test += 1
            if count_test == nb_decoding_test:
                break

/var/folders/bq/ht4rq0pn2yqbdhn46m4mt1d00000gn/T/ipykernel_21749/3550391496.py:34: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  X_train = torch.load(os.path.join(dataset_di

Input: SOS attempting 24 multiple choice questions in 45 minutes is easy so instead of wasting time worrying read the questions carefully this will help you to score better you may even pass with flying colours EOS
Target: SOS manage your time EOS
-----------------------------------
Input: SOS when the popcorn finishes cooking you should have a good amount of popcorn gathered at the bottom of the case use a large spoon or scoop most popcorn makers should have one included to serve the popcorn traditionally theater popcorn is served in paper bags but if you dont have any you can use ordinary bowls most popcorn machines have a number of holes in the bottom of the case for allowing unpopped kernels and very small crumbs to fall into a crumb drawer located below the bottom before serving the popcorn it is advisable to carefully rake the scoop back and forth so that those old maids will fall into the drawer EOS
Target: SOS serve by scooping into bags EOS
-----------------------------------


## Train the model

In [ ]:
hidden_size = 256
max_length = 100
lr = 0.001
weight_decay = 1e-6
batch_size = 128
num_workers = 2
n_epochs = 100
print_example_every = 10
load_checkpoint = False
early_stopping_patience = 5
log_every = 2000

optimizer_hyperparams = {
    'learning_rate': lr,
    'weight_decay': weight_decay,
    'n_epochs': n_epochs,
    'batch_size': batch_size,
    'num_workers': num_workers,
    'early_stopping_patience': early_stopping_patience
}

model_hyperparams = {
    'hidden_size': hidden_size,
    'max_length': max_length
}

In [ ]:
main(root_dir = root_dir,
    model_hyperparams=model_hyperparams,
    tuning = tuning, 
    optimizer_hyperparams=optimizer_hyperparams,
    log_every = log_every,
    print_examples_every=print_example_every,
    load_checkpoint=load_checkpoint,
    name=name,
    legacy=legacy
    )

TypeError: main() missing 1 required positional argument: 'log_every'

### Making inference with the model (on a CPU)

In [ ]:
checkpoint_name = 'best_checkpoint.tar'

In [ ]:
while True:
    input_tensor = input("Enter the text to summarize (or type 'exit' to quit): ")
    if input_tensor.lower() == 'exit':
        break
    # Don't forget to water your plants, they need it to survive.
    inference.main(root_dir, name, checkpoint_name, hidden_size, max_length, input_tensor)

### Hyperparameters tuning

In [ ]:
num_trials = 10

In [ ]:
if HYP_TUNING:
    study = tuning(root_dir, num_trials, name)
    # Save the study results
    study_dir = os.path.join(root_dir, 'parameters_tuning', name, 'study_results')
    os.makedirs(study_dir, exist_ok=True)
    
    # Save the optimization history
    plot_optimization_history(study)
    
    # Save the parameter importances
    plot_param_importances(study)
    
    # Save the slice plot
    plot_slice(study)

### Check training information with tensorboard

In [ ]:
# %load_ext tensorboard
# !tensorboard --logdir='tensorboard_logs'